What we need is a small parcel to work on, with a nice contourplot with critical points and minima found after initiating local method and then a 3d plot. 

In [ ]:
using Globtim
include("../src/lib_func.jl")

# Constants and Parameters
d = 8 # Initial Degree 
const n, a, b = 2, 10, 1 
const scale_factor = a / b       # Scaling factor appears in `main_computation`, maybe it should be a parameter.
const delta, alpha = .9 , 8 / 10  # Sampling parameters
const tol_l2 = 3e-2             # Define the tolerance for the L2-norm

f = HolderTable # Objective function

We need to also return the number of samples used to generate the sample set. It is annoying that the error goes up while the degree has increased.

In [ ]:
while true # Potential infinite loop
    global poly_approx = MainGenerate(f, 2, d, delta, alpha, scale_factor, 0.2) # computes the approximant in Chebyshev basis
    if poly_approx.nrm < tol_l2
        println("attained the desired L2-norm: ", poly_approx.nrm)
        break
    else
        println("current L2-norm: ", poly_approx.nrm)
        println("Number of samples: ", poly_approx.N)
        global d += 1
    end
end

We define the file names

In [ ]:
# Assuming x is already defined as @polyvar x[1:2]
loc = "inputs.ms"
# File path of the output file
file_path_output = "outputs.ms";

We now expand the approximant computed in the tensorized Chebyshev basis into standard monomial basis.

In [ ]:
using DynamicPolynomials, DataFrames
ap = main_nd(n, d, poly_approx.coeffs) 
@polyvar(x[1:n]) # Define polynomial ring 
# Expand the polynomial approximant to the standard monomial basis in the Lexicographic order w.r.t x
names = [x[i].name for i in 1:length(x)]
open(loc, "w") do file
    println(file, join(names, ", "))
    println(file, 0)
end
# Define the polynomial approximant 
PolynomialApproximant = sum(ap .* MonomialVector(x, 0:d))
for i in 1:n
    partial = differentiate(PolynomialApproximant, x[i])
    partial_str = replace(string(partial), "//" => "/")
    open(loc, "a") do file
        if i < n
            println(file, string(partial_str, ","))
        else
            println(file, partial_str)
        end
    end
end

Solve with Msolve

In [ ]:
run(`msolve -v 1 -f inputs.ms -o outputs.ms`)

In [ ]:
function average(X::Vector{Int})::Float64
    return sum(X) / length(X)
end

# Process the file and get the points
evaled = process_output_file(file_path_output)

# Parse the points into correct format
real_pts = []
for pts in evaled
    if typeof(pts)== Vector{Vector{Vector{BigInt}}}
        X = parse_point(pts)
    else 
        X = average.(pts) 
    end
    push!(real_pts, Float64.(X))
end;

Sort through the critical points, make sure they fall into the domain of definition. Make them into a Dataframe.

In [ ]:
condition(point) = -1 < point[1] < 1 && -1 < point[2] < 1
filtered_points = filter(condition, real_pts) # Filter points using the filter function
# Colllect the critical points of the approximant 
h_x = Float64[point[1] for point in filtered_points] # Initialize the x vector for critical points of approximant
h_y = Float64[point[2] for point in filtered_points] # Initialize the y vector
h_z = map(p -> f([p[1], p[2]]), zip(scale_factor * h_x, scale_factor * h_y))
df = DataFrame(x=scale_factor * h_x, y=scale_factor * h_y, z= h_z) # Create a DataFrame

In [ ]:
using PlotlyJS, Colors
# Generate the grid and evaluate the function
N = 60  # resolution of the grid
x = range(-scale_factor, scale_factor, length=N)
y = range(-scale_factor, scale_factor, length=N)
z = [f([xi, yi]) for yi in y, xi in x]

In [ ]:
sf = surface(x=x, y=y, z=z)
# Had to switch the coordinates of the critical points to match the surface plot for some reason. 
crit_pts = scatter3d(x=df.y, y=df.x, z=df.z, mode="markers", marker_size=5, name="Critical Points")

# Layout for the plot
layout = Layout(title="3D Plot of Trefethen function",
    scene=attr(
        xaxis_title="x",
        yaxis_title="y",
        zaxis_title="f(x, y)"),
        height=800
)
# Display the plot layout,
plot([sf,  crit_pts], layout)

Now we want to run some Optimization on the points computed to try to refine them.

In [ ]:
# Optimize the collected entries 
using Optim, LinearAlgebra
for i in 1:nrow(df)
    println("Optimizing for point $i")
    x0 = [df.x[i], df.y[i]]
    try
        res = Optim.optimize(f, x0, LBFGS(), Optim.Options(show_trace=true))
        minimizer = Optim.minimizer(res)
        min_value = Optim.minimum(res)
        steps = res.iterations
        converged = Optim.converged(res)
        distance = norm(x0 - minimizer)

        println(summary(res))
    catch e
        println("Optimization failed for point $i with error: ", e)
    end
end
